# Proyecto EDO Equipo 15

## Integrantes

- Alejandro Camacho Pérez       C-212
- Carlos Arturo Pérez Cabrera   C-212
- Diana Laura Pérez Trujillo    C-212
- David Sánchez Iglesias        C-212

## Ejercicios

Todos los ejercicios son utilizando el libro de Edwards

### Métodos auxiliares y variables globales

In [ ]:
from cairo import Region


EPSILON = 1e-8


def Great_Than(a, b):
    return a - b > 0


def Less_Than(a, b):
    return a - b < 0


def Equal_To(a, b):
    return abs(a - b) < EPSILON


def Euler_Method(function, x,y,max,h):
    coordinates = ([x], [y])

    i = 0
    while(Less_Than(x, max) or Equal_To(x, max)):


        # Update x value
        x = x+h

        # Update y value
        y = y+h*function(x,y)

        # Save values
        coordinates[0].append(x)
        coordinates[1].append(y)

    return coordinates

def Euler_Method_Improved(function, x, y, max, h):
    coordinates = ([x], [y])

    i = 0
    while(Less_Than(x, max) or Equal_To(x, max)):

        # Calculate first slope
        k1 = function(x, y)

        u = y+h*k1

        # Update x value
        x = x+h

        # Calculate second slope
        k2 = function(x, u)

        # Update y value
        y = y+h*(1/2)*(k1+k2)

        # Save values
        coordinates[0].append(x)
        coordinates[1].append(y)

    return coordinates

: 

### Ejercicio 24, página 132

Para el problema se requiere una computadora con
impresora. En este problema de valor inicial utilice el método de Euler mejorado con tamaños de paso h = 0.1, 0.02,
0.004 y 0.0008 para aproximar con 5 cifras decimales el valor
de la solución en 10 puntos igualmente espaciados del intervalo dado. Imprima los resultados en forma tabular con los encabezados apropiados para facilitar la comparación del efecto
de variar el tamaño de paso h. Las primas representan derivadas con respecto a x.

 - $y'= \frac{x}{1+y²},y(-1)=1;-1 \leq x \leq 1$

In [ ]:
from matplotlib import pyplot as pl

def function(x,y):
    return x+(1/5)*y

coordinates = Euler_Method(function=function,x=0,y=-3,max=5,h=1)

pl.plot(coordinates[0],coordinates[1])
pl.show()

### Ejercicio 26, página 132

Suponga que
en un pequeño bosque la población de venados P(t) inicialmente es de 25 individuos y satisface la ecuación
logística  

- $\frac{dP}{dt} = 0.0225P − 0.0003P²$  

(con t en meses). Utilice el método de Euler mejorado con
una calculadora programable o una computadora con el fin
de aproximar la solución para 10 años, primero con un tamaño de paso h = 1 y después con h = 0.5, redondeando
los valores aproximados de P a 3 cifras decimales, ¿qué
porcentaje de la población límite de 75 venados se obtiene
después de 5 años? ¿Después de 10 años?


### Ejercicio 24, Página 142

Suponga
que en un pequeño bosque una población de venados P(t),
inicialmente de 25 individuos, satisface la ecuación logística

- $\frac{dP}{dt} = 0.0225P − 0.0003P²$  

(con t en meses). Aplique el método de Runge-Kutta en
una calculadora programable o en una computadora a fin
de aproximar la solución para 10 años, primero con un tamaño de paso de h = 6 y luego con h = 3, redondeando
los valores aproximados de P a 4 cifras decimales. ¿Qué
porcentaje de la población límite de 75 venados se alcanza después de 5 años? ¿Después de 10 años?
